## PROYECTO PAPIME PE1109232 DESARROLLO DE UN LABORATORIO DE ROBOTICA REMOTA PARA REALIZAR PRACTICAS DE PROGRAMACION DE ALGORITMOS DE PLANEACION Y NAVEGACION EN BANCOS DE PRUEBA FISICOS

# Interfaces (Messages, Services y Actions)

## Resumen
Se presenta una introducción al uso de servicios (Services) y Acciones (Actions), los cuales permiten programar a los robots acciones más complejas al combinar diferentes tipo de información.

Distribución del paquete




## Interfaces

Las interfaces (interfaces), son un conjunto de funcionalidades ROS 2 para la comunicación entre Nodos. Las interfaces de ROS 2 son:

- Mensages: Los mensajes son una funcionalidad la cual define un formato de los Topicos que emplean los Nodos. Los messages permiten comunicas información de manera asincrona. 

- Services: los servicios permiten comunicar información entre Nodos de manera sincrona entre un Nodo client el cual envía un tipo de información y un Nodo service el cual con base en la información recibida por el cliente envía una respuesta específica. 

- Actions: son funcionalidades que se utilizan para indicar a los robots que ejecuten una acción, la estructura de este tipo de funcionalidad es similar a un servicio con retroalimentación la cual permite conocer el estado de la ejecución de una tarea asignada. La retroalimentación permite determinar un cambio en la acción o su cancelación


ROS 2 ya cuenta con interfaces de origen para su uso, para verficar cuales están instaladas se empleta el código:

```txt
$ ros2 interface list

```

Para obsevar los elementos que componen a un interface se puede usar la función de *show*, por ejemplo:

![interface_1.png](images/interface_1.png)



## Messages

Los mensajes son una interface que define la información que transmite un Nodo, por ejemplo:

![interface_2.png](images/interface_2.png)


El mensaje tipo Twist se compone 6 elementos, tres elementos linear del tipo *float64* que se relacionan con la translación de un sistema de referencia específico con respecto a los ejes *x*, *y* y *z* y res elementos angular del tipo *float64* que se relacionan con la orientación en radianes de un sistema del mismo sistema de referencia específico con respecto a los ejes *x*, *y* y *z*.

La mayoría de los tipos de mensajes necesarios se encuentran en la librerias de mensaje de ROS 2.


## Services

ROS 2 cuenta con librerias de servicios asociados con las aplicaciones como Navigation, Moveit, etc. Se presenta la manera de implementar services propios en C++, ya que es la manera standar de implementar este tipo aplicaciones. 

El servicio que se diseño tiene la función de enviar un número y determinar si este par o impar. Para generar el servicio es necesario agregar dentro del paquete la carpeta de servicios *srv*.

![interface_3.png](images/interface_3.png)

Y dentro de la carpeta se agregaran todos los sercivios generados por el usuario, en este caso se crea el servicio *CheckNumber.srv*

```xml
# Request
int64 number # Numero a revsar
---
# Response
string result # Resultado si el numero es par o impar

```

Lo que envía el nodo de servicio es el *Request* que en este caso es un numero del tipo *int64* con la etiqueta *number*, el Nodo que recibe el numero (client) genera la respuesta que entes caso es un mensaje con formato *string*.


# Configuración de Messages y Services propios

Para la utilización de Mensajes y Servicios propios generados por el usaurio se debe crear un metapaquete (carpeta) dentro de este. Por prácticidad se presenta la implementación de Messages y Services en paquete con políticas de CMake, ya que es menos complicado que en paquetes en Python.

## Messages

Para la implementación de mensajes propios se deben seguir los siguientes pasos:


1. Crear un carpeta *msg* (en caso de que el paquete no lo tenga inlcuido).
2. Crear un archivo que contenga los elementos del mesage *<archivo>.msg*, la única restricción que tienen los elementos del mensaje es que deben ser elementos del mismo tipo, por ejemplo:

El archivo *Number.msg* (el nombre de los archivos de mensages y services incluyen mayusculas por norma, aun que sea una composición de palabras, por ejemplo, NumberCheck.msg).

```txt
int64 number_1 
int64 number_2

```
El mensaje se compone de dos elemmentos del tipo *int64*, *number_1* y *number_2*.

3. Configuración del paquete en archivo *CMakeList.txt* y el archivo *package.xml*.

Archivo CMakeKist.txt:

```txt
cmake_minimum_required(VERSION 3.8)
project(srv_act_pkg)

if(CMAKE_COMPILER_IS_GNUCXX OR CMAKE_CXX_COMPILER_ID MATCHES "Clang")
  add_compile_options(-Wall -Wextra -Wpedantic)
endif()

# find dependencies
find_package(ament_cmake REQUIRED)
find_package(rclcpp REQUIRED)
find_package(std_msgs REQUIRED)
# Constructor para las aplicaciones interface
find_package(rosidl_default_generators REQUIRED)

# Generate custom interfaces
rosidl_generate_interfaces(${PROJECT_NAME}
  "msg/Numbers.msg"
  DEPENDENCIES 
  ADD_LINTER_TESTS
)

# Set support for using custom interfaces in C++ code
rosidl_get_typesupport_target(cpp_typesupport_target "${PROJECT_NAME}" "rosidl_typesupport_cpp")


# create executables 
add_executable(message_publisher src/message_publisher.cpp)
ament_target_dependencies(message_publisher rclcpp std_msgs) 
target_link_libraries(message_publisher "${cpp_typesupport_target}")

install(TARGETS
  message_publisher
  DESTINATION lib/${PROJECT_NAME})



install(DIRECTORY
  launch
  DESTINATION share/${PROJECT_NAME}/
)

ament_package()

```

Configuración del archivo *package.xml*

```xml
<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>srv_act_pkg</name>
  <version>0.0.0</version>
  <description>TODO: Package description</description>
  <maintainer email="erik.pena@ingenieria.unam.edu">tsmusr</maintainer>
  <license>MIT</license>

  <buildtool_depend>ament_cmake</buildtool_depend>

  <depend>rclcpp</depend>
  <depend>std_msgs</depend>

  <build_depend>rosidl_default_generators</build_depend>
  <exec_depend>rosidl_default_runtime</exec_depend>
  <member_of_group>rosidl_interface_packages</member_of_group>

  <test_depend>ament_lint_auto</test_depend>
  <test_depend>ament_lint_common</test_depend>

  <export>
    <build_type>ament_cmake</build_type>
  </export>
</package>
```


4. Para utilizar el mensaje en cualquier aplicación de ROS 2 es necesario implementar su ruta dentro del encabezado de un programa, por ejemplo en el siguiente programa message_publisher.cpp:


```cpp
#include "rclcpp/rclcpp.hpp"

#include "srv_act_pkg/msg/numbers.hpp"
#include "std_msgs/msg/string.hpp"

using namespace std::chrono_literals;

class HelloWorldPubNode : public rclcpp::Node
{
  public: 
    HelloWorldPubNode() : Node("hello_world_pub_node") 
    {
      publisher_ = this->create_publisher<std_msgs::msg::String>("hello_world", 10);
      timer_ = this->create_wall_timer(1s, std::bind(&HelloWorldPubNode::publish_hello_world, this));
    }

  private:
    void publish_hello_world()
    {
      auto message = std_msgs::msg::String();
      auto numeros = srv_act_pkg::msg::Numbers();
      numeros.number_1 = 10;
      numeros.number_2 = 3;

      message.data = "Hello World  " + std::to_string(counter_) + "  " + std::to_string(numeros.number_1) + "  " + std::to_string(numeros.number_2);
      publisher_->publish(message);
      counter_++;
    }

    rclcpp::Publisher<std_msgs::msg::String>::SharedPtr publisher_;
    rclcpp::TimerBase::SharedPtr timer_;
    size_t counter_ = 0;
};


int main(int argc, char * argv[])
{
  rclcpp::init(argc, argv);
  rclcpp::spin(std::make_shared<HelloWorldPubNode>());
  rclcpp::shutdown();
  
  return 0;
}

```

La línea *#include "srv_act_pkg/msg/numbers.hpp"* implementa el mensaje el cual se cuentra configurado en el archivo *numbers.hpp*, por lo que es necesario implementar su ruta dentro del paquete en su configuración con "includePath" setting:

![interface_4.png](images/interface_4.png)

Las rutas se incliyen en *One include path per line*

![interface_5.png](images/interface_5.png)


La ruta que se debe agregar es:
```txt
/home/.../<name_ws>/install/<name_pkg>/include/**
```

El paquete utilizar el mensaje custom y el mensaje tipo *srting* de la libreria estandar de ROS 2. 

El publicador reproduce un mensaje, un contador y el valor de los numeros del mensaje configurado con un valor de 10 y 3.

![interface_6.png](images/interface_6.png)

![interface_7.png](images/interface_7.png)


## Services


Para generar los Sercios propios es necesario agreagar el paquete un metapaquete (carpeta) de servicios *srv* para la implementación de estas interfaces. Los archivos *.srv* se componen de dos partes, una que contiene los elementos de una solicitud (request) que un Nodo de servicio (service) le contesta (responce) a un Nodo cliente (client), para ilustrar como se configura e implementa un servicio personalizado. En al carpeta de srv se crea el archivo *CheckNumber.srv*:

```txt
# Request
int64 number # Numero a revsar
---
# Response
string result # Resultado si el numero es par o impar
```
El servicio anterior se implementará en un archivo *service_server.cpp*:

```cpp
#include "rclcpp/rclcpp.hpp"  // Se incluye la biblioteca principal de ROS 2 en C++.
#include "srv_act_pkg/srv/check_number.hpp"  // Se incluye el archivo de servicio que define la estructura del servicio CheckNumber.


// Se define una clase que hereda de rclcpp::Node, la cual representa un nodo de ROS 2.
class CheckNumberServiceNode : public rclcpp::Node
{
    public:
        // Constructor del nodo.
        CheckNumberServiceNode() : Node("check_number_service_node")  // Se inicializa el nodo con el nombre "check_number_service_node".
        {
            // Se crea un servidor de servicio que atenderá las solicitudes del tipo srv_act_pkg::srv::CheckNumber.
            // Se asocia la función miembro check_number_even para procesar cada solicitud.
            service_server_ = this->create_service<srv_act_pkg::srv::CheckNumber>(
                "check_number", std::bind(&CheckNumberServiceNode::check_number_even, this,
             std::placeholders::_1, std::placeholders::_2)
            );
        }

    private:
        // Función de callback que se ejecuta cada vez que se recibe una solicitud.
        // request: Puntero compartido a la solicitud que contiene el número a evaluar.
        // response: Puntero compartido a la respuesta que se enviará al cliente.
        void check_number_even(
            const srv_act_pkg::srv::CheckNumber::Request::SharedPtr request,
            srv_act_pkg::srv::CheckNumber::Response::SharedPtr response)
        {
            // Se calcula el residuo de la división del número ingresado entre 2.
            int remainder = request->number % 2;

            // Se usa un switch para determinar si el número es par o impar.
            switch (remainder)
            {
            case 0:
                // Si el residuo es 0, el número es par.
                response->result = "Par";        
                break;
            
            case 1:
                // Si el residuo es 1, el número es impar.
                response->result = "Impar";
                break;
            }
        }
        
        // Miembro de la clase que almacena el servidor del servicio.
        rclcpp::Service<srv_act_pkg::srv::CheckNumber>::SharedPtr service_server_;
};


// Función principal del programa.
int main(int argc, char * argv[])
{
    // Inicializa el entorno de ROS 2.
    rclcpp::init(argc, argv);
    
    // Se crea una instancia del nodo y se entra en un bucle de espera para procesar las solicitudes.
    rclcpp::spin(std::make_shared<CheckNumberServiceNode>());
    
    // Apaga el entorno de ROS 2 al finalizar.
    rclcpp::shutdown();

    return 0;
}

```

Al incluir la configuración de los mensajes sobre el paquete, la ruta *#include "srv_act_pkg/srv/check_number.hpp"* debe ser reconocida para su configuración.

El siguiente programa crea un Nodo Client el cual proporciona un numero para que sea evaluado por el Nodo Service. El código del Nodo Client es el siguiente:


```cpp
#include "rclcpp/rclcpp.hpp"                // Se incluye la biblioteca principal de ROS 2 en C++.
#include "srv_act_pkg/srv/check_number.hpp" // Se incluye el archivo de servicio que define la estructura del servicio CheckNumber.

// Se utiliza un alias para simplificar el uso del servicio CheckNumber.
typedef srv_act_pkg::srv::CheckNumber CheckNumberSrv;

int main(int argc, char * argv[])
{
    // Inicializa el entorno de ROS 2.
    rclcpp::init(argc, argv);
    
    // Se crea un nodo para el cliente del servicio, con el nombre "check_number_client_node".
    auto service_client_node = rclcpp::Node::make_shared("check_number_client_node");

    // Se crea el cliente del servicio utilizando el nodo, conectándose al servicio "check_number".
    auto client = service_client_node->create_client<CheckNumberSrv>("check_number");

    // Se crea una solicitud del servicio como un objeto compartido.
    auto request = std::make_shared<CheckNumberSrv::Request>();

    // Se solicita al usuario que introduzca un número para determinar si es par o impar.
    std::cout << "Teclea un numero para determinar si es par o impar: ";
    std::cin >> request->number;

    // Se espera hasta que el servicio esté disponible.
    client->wait_for_service();

    // Se envía la solicitud de forma asíncrona y se obtiene una futura respuesta.
    auto response = client->async_send_request(request);

    // Se espera a que la respuesta se procese, comprobando que se haya completado exitosamente.
    if(rclcpp::spin_until_future_complete(service_client_node, response) == rclcpp::FutureReturnCode::SUCCESS)
    {
        // Si la respuesta fue exitosa, se muestra el resultado (Par o Impar) en la salida estándar.
        std::cout << "El numero es " << response.get()->result << std::endl;
    } else {
        // Si ocurre algún error al procesar la respuesta, se informa al usuario.
        std::cout << "Hay un error en el procesamiento de la respuesta";
    }

    // Se cierra el entorno de ROS 2.
    rclcpp::shutdown();

    return 0;
}
```

Explicación del código:

- Inclusión de Librerías: Se incluyen las librerías esenciales de ROS 2 y el archivo que define el servicio CheckNumber, necesario para la comunicación entre nodos.

- Alias para el Servicio: Se utiliza un typedef para simplificar la sintaxis al referirse al tipo srv_act_pkg::srv::CheckNumber como CheckNumberSrv.

- Inicialización y Creación del Nodo: Se inicializa ROS 2 y se crea un nodo específico para el cliente del servicio, que es el encargado de enviar solicitudes.

- Creación y Envío de la Solicitud: Se crea una instancia de solicitud, se le asigna el número introducido por el usuario, y se espera a que el servicio esté disponible para enviar la solicitud de forma asíncrona.

- Procesamiento de la Respuesta: Se utiliza spin_until_future_complete para bloquear la ejecución hasta que la respuesta esté lista. Dependiendo del resultado, se muestra en pantalla si el número es "Par" o "Impar", o un mensaje de error si la operación falla.

- Cierre de ROS 2: Finalmente, se realiza la limpieza y el apagado del entorno de ROS 2 con rclcpp::shutdown().


Se agregan los elementos para el uso de las interfaces del paquete:


```txt
# Se especifica la versión mínima requerida de CMake para este proyecto.
cmake_minimum_required(VERSION 3.8)

# Se define el nombre del proyecto.
project(srv_act_pkg)

# Si el compilador es GNU o Clang, se agregan opciones de compilación para mostrar advertencias (warnings) adicionales.
if(CMAKE_COMPILER_IS_GNUCXX OR CMAKE_CXX_COMPILER_ID MATCHES "Clang")
  add_compile_options(-Wall -Wextra -Wpedantic)
endif()

# Se buscan las dependencias necesarias para el proyecto.
find_package(ament_cmake REQUIRED)
find_package(rclcpp REQUIRED)
find_package(std_msgs REQUIRED)
find_package(rosidl_default_generators REQUIRED)

# Genera las interfaces personalizadas definidas en archivos .srv y .msg.
rosidl_generate_interfaces(${PROJECT_NAME}
  "srv/CheckNumber.srv"  # Archivo de servicio que define la interfaz de CheckNumber.
  "msg/Numbers.msg"      # Archivo de mensaje que define la estructura de Numbers.
  DEPENDENCIES           # Se pueden agregar dependencias adicionales si son necesarias.
  ADD_LINTER_TESTS       # Opción para agregar pruebas de lint a los archivos generados.
)

# Configura el soporte para usar las interfaces personalizadas en código C++.
rosidl_get_typesupport_target(cpp_typesupport_target "${PROJECT_NAME}" "rosidl_typesupport_cpp")

# Se crean los ejecutables del proyecto.

# Ejecutable del servidor de servicio.
add_executable(service_server src/service_server.cpp)
# Se especifican las dependencias del ejecutable, en este caso rclcpp.
ament_target_dependencies(service_server rclcpp) 
# Se vincula el ejecutable con la biblioteca de soporte para las interfaces generadas.
target_link_libraries(service_server "${cpp_typesupport_target}")

# Ejecutable del cliente de servicio.
add_executable(service_client src/service_client.cpp)
ament_target_dependencies(service_client rclcpp) 
target_link_libraries(service_client "${cpp_typesupport_target}")

# Ejecutable del publicador de mensajes.
add_executable(message_publisher src/message_publisher.cpp)
ament_target_dependencies(message_publisher rclcpp std_msgs) 
target_link_libraries(message_publisher "${cpp_typesupport_target}")

# Se especifica la instalación de los ejecutables en el directorio lib del paquete.
install(TARGETS
  service_server
  service_client
  message_publisher
  DESTINATION lib/${PROJECT_NAME}
)

# Se instala el directorio "launch" para facilitar el despliegue del paquete.
install(DIRECTORY
  launch
  DESTINATION share/${PROJECT_NAME}/
)

# Se finaliza la configuración del paquete con ament.
ament_package()

```

En el caso de los metadatos el *package.xml*:

```xml
<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>srv_act_pkg</name>
  <version>0.0.0</version>
  <description>TODO: Package description</description>
  <maintainer email="erik.pena@ingenieria.unam.edu">tsmusr</maintainer>
  <license>MIT</license>

  <buildtool_depend>ament_cmake</buildtool_depend>

  <depend>rclcpp</depend>
  <depend>std_msgs</depend>

  <build_depend>rosidl_default_generators</build_depend>
  <exec_depend>rosidl_default_runtime</exec_depend>
  <member_of_group>rosidl_interface_packages</member_of_group>

  <test_depend>ament_lint_auto</test_depend>
  <test_depend>ament_lint_common</test_depend>

  <export>
    <build_type>ament_cmake</build_type>
  </export>
</package>

```



Al volver a construir el paquete y refrescar la fuente de este se puede utilizar el servicio:


![interface_8.png](images/interface_8.png)



